In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('./drive/My Drive/wjsfur/train.csv')

In [ ]:
train.head()

In [ ]:
train_list = []
for i in range(1,101):
    train_list.append(train[train['건물번호']==i])
df = train_list[0]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def visual_df(df):
    # 일시(datetime) 변환
    df['일시'] = pd.to_datetime(df['일시'], format='%Y%m%d %H')

    # 전력소비량 시각화
    plt.figure(figsize=(14, 5))
    plt.plot(df['일시'], df['전력소비량(kWh)'], color='tab:blue')
    plt.title("Hourly Power Consumption", fontsize=16)
    plt.xlabel("Time")
    plt.ylabel("Power Consumption (kWh)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()

In [ ]:
for df in train_list:
    print(set(df['건물번호']))
    visual_df(df)

## 이상치 탐지

In [ ]:
from statsmodels.tsa.seasonal import STL
import numpy as np
import pandas as pd

for i in range(len(train_list)):
    # 1) 복사본 생성하여 SettingWithCopyWarning 방지
    df = train_list[i].copy()
    original_length = len(df)

    # -------------------------
    # 2) 전력소비량이 0인 값을 NaN으로 변경 (완전 제거 대신)
    # -------------------------
    zero_mask = df['전력소비량(kWh)'] == 0
    df.loc[zero_mask, '전력소비량(kWh)'] = np.nan
    zero_count = zero_mask.sum()

    # 유효한 데이터만으로 이상치 탐지 진행
    df_valid = df.dropna(subset=['전력소비량(kWh)']).reset_index(drop=True)

    # -------------------------
    # 3) STL 기반 이상치 탐지
    # -------------------------
    if len(df_valid) < 48:  # 최소 2일치 데이터 필요
        print(f"건물번호 {df['건물번호'].iloc[0]} → 데이터 부족으로 STL 생략, 선형 보간만 적용")
        # 0값들을 선형 보간으로 처리
        df = df.sort_values('일시').reset_index(drop=True)
        df['전력소비량(kWh)'] = df['전력소비량(kWh)'].interpolate(method='linear')
        df['전력소비량(kWh)'] = df['전력소비량(kWh)'].fillna(method='bfill').fillna(method='ffill')

        train_list[i] = df
        continue

    # 인덱스를 일시로 설정하되 중복 제거
    df_temp = df_valid.set_index('일시')
    df_temp = df_temp[~df_temp.index.duplicated(keep='first')]  # 중복 인덱스 제거
    series = df_temp['전력소비량(kWh)']

    try:
        stl = STL(series, period=24)  # 하루 주기
        res = stl.fit()

        resid = res.resid
        z = (resid - resid.mean()) / resid.std()
        outliers_stl_idx = series.index[np.abs(z) > 3]   # STL + z-score 이상치 인덱스

    except Exception as e:
        print(f"건물번호 {df['건물번호'].iloc[0]} → STL 처리 오류: {e}")
        outliers_stl_idx = pd.Index([])

    # -------------------------
    # 4) IQR 기반 이상치 탐지
    # -------------------------
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_iqr_idx = series.index[(series < lower_bound) | (series > upper_bound)]

    # -------------------------
    # 5) 두 방식 결과 합치기
    # -------------------------
    all_outlier_idx = pd.Index(list(set(outliers_stl_idx) | set(outliers_iqr_idx)))

    # -------------------------
    # 6) 이상치를 NaN으로 변경 (제거하지 않고 보간 준비)
    # -------------------------
    outlier_mask = df['일시'].isin(all_outlier_idx)
    df.loc[outlier_mask, '전력소비량(kWh)'] = np.nan
    outliers_count = outlier_mask.sum()

    # -------------------------
    # 7) 보간법으로 결측값 채우기
    # -------------------------
    # 시계열 순서 보장을 위해 일시 기준으로 정렬
    df = df.sort_values('일시').reset_index(drop=True)

    # 선형 보간 적용
    df['전력소비량(kWh)'] = df['전력소비량(kWh)'].interpolate(method='linear')

    # 시작이나 끝에 NaN이 남아있으면 앞/뒤 값으로 채우기
    df['전력소비량(kWh)'] = df['전력소비량(kWh)'].fillna(method='bfill').fillna(method='ffill')

    # 그래도 NaN이 남아있으면 평균값으로 채우기
    if df['전력소비량(kWh)'].isna().any():
        mean_value = df['전력소비량(kWh)'].mean()
        df['전력소비량(kWh)'] = df['전력소비량(kWh)'].fillna(mean_value)

    # train_list 갱신
    train_list[i] = df

    # 결과 확인용 출력
    total_interpolated = zero_count + outliers_count
    print(f"건물번호 {df['건물번호'].iloc[0]} → STL:{len(outliers_stl_idx)}개, IQR:{len(outliers_iqr_idx)}개")
    print(f"  - 0값:{zero_count}개, 이상치:{outliers_count}개, 총 보간처리:{total_interpolated}개")
    print(f"  - 최종 데이터 길이: {len(df)}개 (원본: {original_length}개)")

In [ ]:
cnt=0
for df in train_list:
    cnt += len(df)
204000 - cnt

In [ ]:
for df in train_list:
    visual_df(df)